In [130]:
from bs4 import BeautifulSoup
import pandas as pd
import pickle
import numpy as np
import missingno as msno
import time, re
from bs4 import BeautifulSoup
import requests


In [181]:
gecko = pd.read_csv("gecko_update.csv")
gecko["fully_diluted_valuation"] = np.nan
gecko["website"] = "?"
gecko["explorer"] = "?"
gecko["contract_addr"] = "?"
gecko["twitter_url"] = "?"
gecko["tag"] = "?"
total_crypto_mkt_cap = 1521620816260
url = "https://www.coingecko.com/en/coins/bscshiba"
USER_AGENT = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36"
headers = {'user-agent': USER_AGENT}

print(f"Scraping {url}\n")
response = requests.get(url, headers=headers)
print(response.status_code)
coin_html_text = response.text
coin_soup = BeautifulSoup(coin_html_text, 'lxml')

gecko.head()

Scraping https://www.coingecko.com/en/coins/bscshiba

200


,Unnamed: 0,name,symbol,price,volume,mkt_cap,coingecko_rank,vol / mkt_cap,dominance,num_holders,whitepaper,fully_diluted_valuation,website,explorer,contract_addr,twitter_url,tag
0,0,Bitcoin,BTC,NaN,2.373029e+10,6.323045e+11,1.0,0.038,0.415547,NaN,NaN,NaN,?,?,?,?,?
1,1,Ethereum,ETH,NaN,2.285632e+10,2.492309e+11,2.0,0.092,0.163793,NaN,1.0,NaN,?,?,?,?,?
2,2,Tether,USDT,NaN,4.973612e+10,6.252556e+10,3.0,0.795,0.041091,NaN,1.0,NaN,?,?,?,?,?
3,3,Binance Coin,BNB,NaN,1.198426e+09,4.860686e+10,4.0,0.025,0.031944,NaN,NaN,NaN,?,?,?,?,?
4,4,Cardano,ADA,NaN,1.463115e+09,4.304068e+10,5.0,0.034,0.028286,NaN,1.0,NaN,?,?,?,?,?


In [184]:
div_RHS = "body > div.container > div.mt-3 > div.col-12.row.p-0.m-0.mb-2.tw-flex.flex-column-reverse.flex-sm-row > div.col-md-3.col-lg-5.text-center.text-md-right.mt-3.mt-md-0.pr-0 > div"
div_LHS = "body > div.container > div.mt-3 > div.col-12.row.p-0.m-0.mb-2.tw-flex.flex-column-reverse.flex-sm-row > div.col-md-9.col-lg-7.p-0 > div"

table_1 = coin_soup.select(div_LHS)[0]
try:
    table_2 = coin_soup.select(div_RHS)[0]

    ## Determine if table_2 has value for fully diluted valuation or circ supply
    fully_diluted_valuation = np.nan
    table_2_categories = table_2.find_all("div", class_="font-weight-bold")
    has_fully_diluted_valuation = False
    idx = 0
    for tag in table_2_categories:
        label = tag.text.strip()
        if label == "Fully Diluted Valuation":
            has_fully_diluted_valuation = True
            break
        idx += 1

    if has_fully_diluted_valuation:
        table_2_values = table_2.find_all("div",class_="mt-1")
        fully_diluted_valuation = float(table_2_values[idx].span.text.strip().replace(
            ",", "").replace("$",""))
    else:
        idx = 0
        has_circ_supply = False
        for tag in table_2_categories:
            label = tag.text.strip()
            if label == "Circulating Supply":
                has_circ_supply = True
                break
            idx += 1
        
        table_2_values = table_2.find_all("div",class_="mt-1")
        try:
            fully_diluted_valuation = float(table_2_values[idx].text.split("/")[1].strip().replace(",", ""))
        except Exception:
            fully_diluted_valuation = np.nan

    gecko.at[0, 'fully_diluted_valuation'] = fully_diluted_valuation
except:
    print("Table 2 does not exist")
## Get website url, twitter url, check for whitepaper, explorer url, tag
table_1_categories = table_1.find_all("span", class_="coin-link-title")
table_1_rows = table_1.find_all("div", class_="coin-link-row")
category_dict = {"web": (False, 0),
                "explorer": (False, 0),
                "tag": (False, 0),
                "contract": (False, 0),
                "community": (False, 0)}

idx = 0
for category in table_1_categories:
    if category.text.strip() == "Website":
        category_dict['web'] = (True, idx)
    elif category.text.strip() == "Explorers":
        category_dict['explorer'] = (True, idx)
    elif category.text.strip() == "Contract":
        category_dict['contract'] = (True, idx)
    elif category.text.strip() == "Community":
        category_dict['community'] = (True, idx)
    elif category.text.strip() == "Tags":
        category_dict['tag'] = (True, idx)
    idx += 1

#print(table_1_rows[2], "\n\n")

for k, v in category_dict.items():
    if k == "web" and v[0]:
        web_row = table_1_rows[v[1]]
        web_row_links = web_row.find_all("a")
        if web_row_links:
            # Check for link to whitepaper
            for r in web_row_links:
                if r.text == "Whitepaper":
                    gecko.at[0, "whitepaper"] = 1
            gecko.at[0, "website"] = web_row_links[0]['href']
    elif k == "explorer" and v[0]:
        exp_row = table_1_rows[v[1]]
        gecko.at[0, "explorer"] = exp_row.find("a")['href']
    elif k == "contract" and v[0]:
        contract_row = table_1_rows[v[1]]
        # Sometimes contract category idx and correct div don't line up exactly
        try:
            gecko.at[0, "contract_addr"] = contract_row.find("i")['data-address']
        except:
            contract_row = table_1_rows[v[1]+1]
            gecko.at[0, "contract_addr"] = contract_row.find("i")['data-address']
    elif k == "community" and v[0]:
        community_row = table_1_rows[v[1]]
        community_links = community_row.find_all("a")
        for l in community_links:
            if l.text == "Twitter":
                gecko.at[0, "twitter_url"] = l['href']
    elif k == "tag" and v[0]:
        tag_row = table_1_rows[v[1]]
        gecko.at[0, "tag"] = tag_row.div.findChildren()[0].text


Table 2 does not exist
<div class="coin-link-row mb-md-0" data-controller="coin-contract-address">
<span class="coin-link-title mr-2">Contract</span>
<div class="tw-flex flex-wrap">
<div class="tw-px-2.5 tw-flex tw-items-center tw-justify-center tw-py-1 tw-h-7 tw-my-0.5 tw-rounded-md tw-rounded-r-none tw-text-sm tw-font-medium tw-bg-gray-100 tw-text-gray-800 dark:tw-text-white dark:tw-bg-gray-600 dark:tw-bg-opacity-60">
<span class="text-muted mr-2">
<span data-content="BSC" data-delay='{"show":0,"hide":300}' data-html="true" data-placement="top" data-toggle="popover" data-trigger="focus" tabindex="0"><img height="20px" src="https://assets.coingecko.com/coins/images/825/small/binance-coin-logo.png?1547034615" width="20px"/></span>
</span>
<span>0x615...bef3e</span>
<i class="far tw-text-sm tw-ml-2 align-middle hover:tw-bg-gray-200 dark:hover:tw-bg-gray-500 fa-far fa-clone" data-action="mousedown-&gt;coin-contract-address#copyAddress
        mouseenter-&gt;coin-contract-address#changeCu

In [163]:
coin_soup.find()

<div class="tab-pane" id="social">
<div class="card-block">
<div class="text-muted text-center p-4">
Loading...
</div>
</div>
</div>

In [13]:
gecko = pd.read_csv("coingecko.csv")
holders = pd.read_csv("holders.csv")

gecko["num_holders"] = np.nan
gecko_names = [name.split("(")[0].strip().lower().replace(" ", "_") for name in gecko["name"]]
holders["symbol"] = [name.split("(")[1].strip().replace(")", "") for name in holders["name"]]
gecko

,Unnamed: 0,name,symbol,price,volume,mkt_cap,coingecko_rank,vol / mkt_cap,dominance,num_holders
0,0,Bitcoin,BTC,NaN,2.373029e+10,6.323045e+11,1.0,0.038,0.415547,NaN
1,1,Ethereum,ETH,NaN,2.285632e+10,2.492309e+11,2.0,0.092,0.163793,NaN
2,2,Tether,USDT,NaN,4.973612e+10,6.252556e+10,3.0,0.795,0.041091,NaN
3,3,Binance Coin,BNB,NaN,1.198426e+09,4.860686e+10,4.0,0.025,0.031944,NaN
4,4,Cardano,ADA,NaN,1.463115e+09,4.304068e+10,5.0,0.034,0.028286,NaN
...,...,...,...,...,...,...,...,...,...,...
8395,8395,Fox Finance,FOX,NaN,1.273278e+04,NaN,NaN,NaN,NaN,NaN
8396,8396,Morty Token,MRTY,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8397,8397,Wrapped Huobi Token,WHT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8398,8398,Thinkium,TKM,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN


In [100]:
holders

,Unnamed: 0,name,num_holders,symbol
0,0,Tether USD (USDT),3267088,USDT
1,1,BNB (BNB),315730,BNB
2,2,USD Coin (USDC),973375,USDC
3,3,HEX (HEX),222504,HEX
4,4,Uniswap (UNI),251495,UNI
...,...,...,...,...
977,977,Speed Mining Service (SMS),185,SMS
978,978,ATLANT (ATL),6742,ATL
979,979,Zilla (ZLA),14311,ZLA
980,980,eBitcoin (EBTC),4978,EBTC


In [44]:
for index, row in holders.iterrows():
    name = row["name"].split("(")[0].strip().lower().replace(" ", "_")
    symbol = row["symbol"]
    #print(name, symbol)
    if name in gecko_names and symbol in gecko['symbol'].tolist():
        idx = gecko_names.index(name)
        gecko.at[idx, "num_holders"] = row["num_holders"]
        


True     8005
False     395
Name: num_holders, dtype: int64


In [47]:
gecko.to_csv("gecko_holders.csv")

In [52]:
gecko["whitepaper"] = np.nan
papers = pd.read_csv("whitepapers.csv", names=['names'])
papers["names"] = papers["names"].str.strip().str.lower().str.replace(" ", "")
for name in papers["names"].tolist():
    if name in gecko_names:
        idx = gecko_names.index(name)
        gecko.at[idx, "whitepaper"] = 1

gecko['whitepaper'].isna().value_counts()

True     7381
False    1019
Name: whitepaper, dtype: int64

In [53]:
gecko.to_csv("gecko_holders_whitepaper.csv")

In [112]:
gecko = pd.read_csv("gecko_holders_whitepaper.csv", usecols=[
    'name',"symbol", "price","volume", "mkt_cap","coingecko_rank", 
    "vol / mkt_cap", "dominance", "whitepaper", "num_holders"])

for index, row in holders.iterrows():
    name = row["name"].split("(")[0].strip().lower().replace(" ", "_")
    symbol = row["symbol"]
    #print(name, symbol)
    if name in gecko_names and symbol in gecko['symbol'].tolist():
        idx = gecko_names.index(name)
        gecko.at[idx, "num_holders"] = row["num_holders"]

gecko.to_csv("gecko_update.csv")

In [205]:
gecko = pd.read_csv("gecko_final_4.csv")
print(gecko["is_scam"].notna().value_counts())
shitcoins = pd.read_csv("deadcoins.csv", sep=";", usecols=[0])

for index, row in shitcoins.iterrows():
    name = row["full_name"].strip().lower().replace(" ", "_")
    if name in gecko_names:
        idx = gecko_names.index(name)
        gecko.at[idx, "is_scam"] = 1



print(gecko["is_scam"].notna().value_counts())
gecko.to_csv("gecko_final_5.csv")



False    8185
True      215
Name: is_scam, dtype: int64
False    8052
True      348
Name: is_scam, dtype: int64


In [199]:
gecko['whitepaper'].isna().value_counts()

True     6412
False    1988
Name: whitepaper, dtype: int64

In [214]:
gecko = pd.read_csv("gecko_final_5.csv")

url = "https://99bitcoins.com/deadcoins/"
USER_AGENT = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36"
headers = {'user-agent': USER_AGENT}

print(f"Scraping {url}\n")
response = requests.get(url, headers=headers)
print(response.status_code)

deadcoin_html_text = response.text
deadcoin_soup = BeautifulSoup(deadcoin_html_text, 'lxml')
rows = deadcoin_soup.find("tbody", class_="list").find_all("tr")
for r in rows:
    name = r.find("td", class_="nnbitcoins-deadcoins-list-item-name").text
    name = name.strip().lower().replace(" ", "_")
    if name in gecko_names:
        idx = gecko_names.index(name)
        gecko.at[idx, "is_scam"] = 1


print(gecko["is_scam"].notna().value_counts())
gecko.to_csv("gecko_final_6.csv")

Scraping https://99bitcoins.com/deadcoins/

200
False    7971
True      429
Name: is_scam, dtype: int64


In [221]:
gecko = pd.read_csv("gecko_final_6.csv", usecols=range(5,23))
gecko.to_csv("gecko_final_7.csv", index=False)
